## Testing the New Models

### Meyer Dataset

#### Reconstruction Property

Imports

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt

Data

In [2]:
data=np.load('data/dataset_meyer/dataset_meyer_test_256_100.npz')
n=data['density']
f=data['F']
e=data['energy']
n_torch=torch.from_numpy(n)[-100:]
f_torch=torch.from_numpy(f)[-100:]
e_torch=torch.from_numpy(e)[-100:]
model=torch.load('model_dft_pytorch/meyer_case_2ndGEN/DFTVAEnorm2ndGEN_hidden_channels_vae_[60, 60, 60, 60, 60]_hidden_neurons_dft_[60, 60, 60, 60, 60]_kernel_size_13_pooling_size_2_latent_dimension_8_l1_0.5_l2_0.001',map_location='cpu')
#model=torch.load('model_dft_pytorch/meyer_case/meyer_model_60_hc_13_ks_2_ps_[40, 40, 40, 40, 40]_hidden_neurons_16_ls_0.0_alpha_0.01_vb',map_location='cpu')
model.eval()
model=model.double()


Reconstruction Test

In [3]:
n_torch = n_torch.unsqueeze(1)
latent_mu, latent_logvar = model.Encoder(n_torch)
n_recon = model.Decoder(latent_mu)
dx=1/256
print(n_recon.shape)
dn=torch.sum(torch.abs(n_torch.squeeze(1)-n_recon.squeeze(1)),dim=-1)*dx
dn=dn.mean()
print(dn.item())

torch.Size([100, 1, 256])
0.0012555295432945248


Visualization

In [ ]:
n_recon_img=n_recon.squeeze(1).detach().cpu().numpy()
n_img=n_torch.squeeze(1).detach().cpu().numpy()

for i in range(3):
    plt.plot(n_recon_img[i])
    plt.plot(n_img[i])
    plt.show()

Prediction Test (MODEL B)

In [4]:
print(n_torch.shape)
latent_mu, latent_logvar = model.Encoder(n_torch)
f_ml=model.DFTModel(latent_mu).view(-1)

df=torch.abs(f_torch-f_ml)/torch.abs(f_ml)

print(df.mean()*627,df.std()*627)


torch.Size([100, 1, 256])
tensor(0.1453, dtype=torch.float64, grad_fn=<MulBackward0>) tensor(0.1362, dtype=torch.float64, grad_fn=<MulBackward0>)


Prediction Test (MODEL A)

In [13]:

f_ml=model.DFTModel(n_torch).view(-1)

df=torch.abs(f_torch-f_ml)/torch.abs(f_ml)

print(df.mean()*627,df.std()*627)

tensor(0.1497, dtype=torch.float64) tensor(0.1421, dtype=torch.float64)
